# BCG Multi-Agent & Multimodal AI Platform - Multi-Agent System

This notebook demonstrates the multi-agent system of the BCG Multi-Agent & Multimodal AI Platform, including:
1. Setting up specialized agents with different roles
2. Orchestrating agent collaboration using LangGraph
3. Testing the complete multi-agent system with complex queries
4. Analyzing agent interactions and workflow patterns

In [ ]:
import os
import sys
import logging
from pathlib import Path

# Add the project root to the Python path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

## 1. Setup and Configuration

In [ ]:
from configs.config import (
    PROCESSED_DATA_DIR,
    LLM_MODEL,
    OLLAMA_BASE_URL,
)

# Check if Ollama is available
import requests
try:
    response = requests.get(f"{OLLAMA_BASE_URL}/api/tags")
    if response.status_code == 200:
        print(f"Ollama is available at {OLLAMA_BASE_URL}")
        models = response.json().get("models", [])
        if models:
            print(f"Available models: {[model['name'] for model in models]}")
        else:
            print("No models found in Ollama")
    else:
        print(f"Ollama API returned status code {response.status_code}")
except Exception as e:
    print(f"Error connecting to Ollama: {str(e)}")
    print(f"Please ensure Ollama is running at {OLLAMA_BASE_URL} with the {LLM_MODEL} model loaded")

## 2. Initialize Language Model

First, let's initialize the language model that will be used by all agents.

In [ ]:
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI

# Initialize Ollama LLM
llm = Ollama(
    model=LLM_MODEL,
    temperature=0.2,
    base_url=OLLAMA_BASE_URL,
)

# Uncomment to use OpenAI instead
# llm = ChatOpenAI(
#     model_name="gpt-4-turbo",
#     temperature=0.2,
# )

# Test the LLM
test_response = llm.invoke("What is BCG known for?")
print("Test LLM response:")
print(test_response)

## 3. Set Up Directories for Visual Analysis

We need to set up directories for visual elements that will be analyzed by the visual analyst agent.

In [ ]:
# Find visual elements in processed data
visuals_directory = None

# Check for visuals in processed documents
for doc_dir in PROCESSED_DATA_DIR.iterdir():
    if doc_dir.is_dir():
        visuals_dir = doc_dir / "visuals"
        if visuals_dir.exists() and visuals_dir.is_dir():
            visuals_directory = visuals_dir
            break

if visuals_directory:
    print(f"Found visuals directory: {visuals_directory}")
    visual_files = list(visuals_directory.glob("*.png")) + list(visuals_directory.glob("*.jpg"))
    print(f"Found {len(visual_files)} visual files")
else:
    print("No visuals directory found. Visual analysis capabilities will be limited.")

## 4. Initialize Individual Agents

Let's initialize and test each specialized agent individually.

In [ ]:
from src.agents.researcher_agent import ResearcherAgent
from src.agents.analyst_agent import AnalystAgent
from src.agents.visual_analyst_agent import VisualAnalystAgent
from src.agents.strategist_agent import StrategistAgent
from src.agents.critic_agent import CriticAgent

# Initialize researcher agent
researcher_agent = ResearcherAgent(
    llm=llm,
    verbose=True,
)

# Test researcher agent
research_query = "What are BCG's carbon emission reduction targets?"
print(f"Testing researcher agent with query: '{research_query}'")
research_result = researcher_agent.research(research_query)
print("\nResearcher agent response:")
print(research_result[:500] + "..." if len(research_result) > 500 else research_result)

In [ ]:
# Initialize analyst agent
analyst_agent = AnalystAgent(
    llm=llm,
    verbose=True,
)

# Test analyst agent
analysis_query = "Analyze BCG's sustainability metrics over the last three years"
data_to_analyze = research_result  # Use the research result as data
print(f"Testing analyst agent with query: '{analysis_query}'")
analysis_result = analyst_agent.analyze(data_to_analyze, analysis_query)
print("\nAnalyst agent response:")
print(analysis_result[:500] + "..." if len(analysis_result) > 500 else analysis_result)

In [ ]:
# Initialize visual analyst agent
visual_analyst_agent = VisualAnalystAgent(
    llm=llm,
    visuals_directory=str(visuals_directory) if visuals_directory else None,
    verbose=True,
)

# Test visual analyst agent
if visuals_directory:
    visual_query = "Find charts related to carbon emissions"
    print(f"Testing visual analyst agent with query: '{visual_query}'")
    visual_result = visual_analyst_agent.analyze_visual(description=visual_query)
    print("\nVisual analyst agent response:")
    print(visual_result[:500] + "..." if len(visual_result) > 500 else visual_result)
    
    # If the visual analyst found any images, analyze the first one
    import re
    image_matches = re.findall(r"\d+\. ([\w\.-]+)\n", visual_result)
    if image_matches:
        image_name = image_matches[0]
        print(f"\nAnalyzing image: {image_name}")
        image_analysis = visual_analyst_agent.analyze_visual(image_path=image_name, description="Analyze this chart")
        print("\nImage analysis result:")
        print(image_analysis[:500] + "..." if len(image_analysis) > 500 else image_analysis)
else:
    print("Skipping visual analyst test due to missing visuals directory")

In [ ]:
# Initialize strategist agent
strategist_agent = StrategistAgent(
    llm=llm,
    verbose=True,
)

# Test strategist agent
inputs = {
    "Research": research_result,
    "Analysis": analysis_result,
}

if 'visual_result' in locals():
    inputs["Visual Analysis"] = visual_result

print(f"Testing strategist agent with {len(inputs)} inputs")
strategy_result = strategist_agent.synthesize(inputs)
print("\nStrategist agent response:")
print(strategy_result[:500] + "..." if len(strategy_result) > 500 else strategy_result)

In [ ]:
# Initialize critic agent
critic_agent = CriticAgent(
    llm=llm,
    verbose=True,
)

# Test critic agent
print(f"Testing critic agent")
critique_result = critic_agent.evaluate(strategy_result, research_result)
print("\nCritic agent response:")
print(critique_result[:500] + "..." if len(critique_result) > 500 else critique_result)

## 5. Initialize and Test Multi-Agent System

Now, let's initialize and test the complete multi-agent system.

In [ ]:
from src.agents.agent_graph import BCGMultiAgentSystem

# Initialize the multi-agent system
multi_agent_system = BCGMultiAgentSystem(
    llm=llm,
    visuals_directory=str(visuals_directory) if visuals_directory else None,
    verbose=True,
)

print("BCG Multi-Agent System initialized successfully")

## 6. Test Multi-Agent System with Simple Query

In [ ]:
# Process a simple query
simple_query = "What progress has BCG made in reducing its carbon emissions in the last few years?"
print(f"Processing query: '{simple_query}'")

simple_response = multi_agent_system.process_query(simple_query)

print("\nMulti-Agent System Response:")
print(simple_response)

## 7. Test Multi-Agent System with Complex Query

Let's test the system with a more complex query that requires multiple agents to collaborate.

In [ ]:
# Process a complex query
complex_query = """How has BCG's approach to sustainability evolved over the past three years, and what are the key metrics and trends in their environmental impact? Also, what strategic recommendations would you make for them to further improve their sustainability performance?"""
print(f"Processing query: '{complex_query}'")

complex_response = multi_agent_system.process_query(complex_query)

print("\nMulti-Agent System Response:")
print(complex_response)

## 8. Test Multi-Agent System with Visual Analysis Query

Let's test the system with a query that specifically requires visual analysis.

In [ ]:
# Process a query requiring visual analysis
if visuals_directory:
    visual_query = "What do BCG's charts and graphs reveal about their progress in sustainability metrics? Can you analyze any visual data from their reports and explain the trends?"
    print(f"Processing query: '{visual_query}'")

    visual_response = multi_agent_system.process_query(visual_query)

    print("\nMulti-Agent System Response:")
    print(visual_response)
else:
    print("Skipping visual analysis test due to missing visuals directory")

## 9. Test Multi-Agent System with Data Analysis Query

Let's test the system with a query that requires detailed data analysis.

In [ ]:
# Process a query requiring data analysis
data_query = "Analyze the trends in BCG's diversity and inclusion metrics over time. What patterns do you see and what conclusions can you draw?"
print(f"Processing query: '{data_query}'")

data_response = multi_agent_system.process_query(data_query)

print("\nMulti-Agent System Response:")
print(data_response)

## 10. Test Multi-Agent System with Expert Query

Let's test the system with a query that requires providing expert strategic recommendations.

In [ ]:
# Process a query requiring strategic recommendations
expert_query = "Based on BCG's sustainability reports, what strategic initiatives should they prioritize over the next 3-5 years to become a leader in sustainability? Provide specific, actionable recommendations."
print(f"Processing query: '{expert_query}'")

expert_response = multi_agent_system.process_query(expert_query)

print("\nMulti-Agent System Response:")
print(expert_response)

## 11. Analyzing Agent Collaboration

Let's analyze how different agents collaborate and contribute to the final response.

In [ ]:
from IPython.display import Markdown, display
import networkx as nx
import matplotlib.pyplot as plt

# Create a simplified representation of the agent workflow
G = nx.DiGraph()

# Add nodes for each agent
nodes = [
    "Coordinator",
    "Researcher",
    "Analyst",
    "Visual Analyst",
    "Strategist",
    "Critic",
    "Finalizer"
]

for node in nodes:
    G.add_node(node)

# Add edges representing possible flows
edges = [
    ("Coordinator", "Researcher"),
    ("Coordinator", "Analyst"),
    ("Coordinator", "Visual Analyst"),
    ("Researcher", "Analyst"),
    ("Researcher", "Visual Analyst"),
    ("Researcher", "Strategist"),
    ("Analyst", "Visual Analyst"),
    ("Analyst", "Strategist"),
    ("Visual Analyst", "Strategist"),
    ("Strategist", "Critic"),
    ("Critic", "Researcher"),
    ("Critic", "Analyst"),
    ("Critic", "Visual Analyst"),
    ("Critic", "Strategist"),
    ("Critic", "Finalizer")
]

for edge in edges:
    G.add_edge(edge[0], edge[1])

# Create a visualization of the agent workflow
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G, k=0.5, seed=42)
nx.draw(G, pos, with_labels=True, node_color="skyblue", node_size=2000, 
        font_size=12, font_weight="bold", arrows=True, arrowsize=20, 
        arrowstyle="->", width=2, edge_color="gray")
plt.title("BCG Multi-Agent Workflow", size=16)
plt.tight_layout()
plt.show()

# Display agent roles and responsibilities
agent_descriptions = """
| Agent | Role | Responsibilities |
|-------|------|------------------|
| **Coordinator** | Query Router | Analyzes the query and determines which specialist agent should handle it first |
| **Researcher** | Information Gatherer | Collects relevant information from BCG reports and external sources |
| **Analyst** | Data Analyst | Analyzes data, identifies trends, and generates insights |
| **Visual Analyst** | Visual Interpreter | Analyzes charts, graphs, and visual elements to extract insights |
| **Strategist** | Strategic Advisor | Synthesizes information and formulates strategic recommendations |
| **Critic** | Quality Evaluator | Evaluates outputs, identifies issues, and suggests improvements |
| **Finalizer** | Response Generator | Creates the final comprehensive response incorporating all insights |
"""

display(Markdown(agent_descriptions))

## 12. Conclusion

In this notebook, we've successfully demonstrated the multi-agent system of the BCG Multi-Agent & Multimodal AI Platform. We've shown how multiple specialized agents can collaborate using LangGraph to provide comprehensive, thoughtful responses to complex queries about BCG's sustainability efforts.

Key accomplishments include:
1. Setting up specialized agents with different roles and capabilities
2. Orchestrating agent collaboration through a flexible workflow
3. Testing the system with various types of queries
4. Analyzing the agent collaboration patterns

This multi-agent architecture allows for more sophisticated reasoning, fact-checking, and strategic thinking than a single LLM could provide alone. The system's modular design also allows for future enhancements, such as adding new specialized agents or improving the existing ones with more advanced capabilities.